In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [2]:
import json
with open("service1906_1506.json") as of:
    data = json.load(of)

In [3]:
computes = [c for c in data.keys() if c!="timespan"]
variables = [v for v in data[computes[0]] if v!='index' and v!='arrJob_scheduling']

In [4]:
#Check empty array
def getEmptyArr(data, c):
    cObj = data[c]
    cDf = pd.DataFrame()
    cDf['compute'] = [c for _ in data['timespan']]
    cDf['timespan'] = data['timespan']
    for v in variables:
        vArr = np.array(cObj[v])
        if len(vArr)==0:
            print('c=', c)
            print('v=', v)
for c in computes:
    getEmptyArr(data, c)

c= compute-1-26
v= arrTemperature


In [5]:
def addTarget(cDf, predictedVar, predictedStep, target):
    cDf[target] = cDf[predictedVar].shift(-predictedStep)
    cDf.dropna(inplace=True)
    return cDf

In [6]:
def getComputeDf(data, c, predictedVar, predictedStep, target):
    cObj = data[c]
    cDf = pd.DataFrame()
    cDf['compute'] = [c for _ in data['timespan']]
    cDf['timespan'] = data['timespan']
    for v in variables:
        vArr = np.array(cObj[v])
        if len(vArr)==0:
            return None
        else:
            for i in range(len(vArr[0])):
                cDf[v+str(i)] = vArr[:, i]
    cDf['timespan'] = pd.to_datetime(cDf['timespan'])
    return addTarget(cDf, predictedVar, predictedStep, target)

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [8]:
def splitTrainTest(predictedVar):
    target = predictedVar + "_target"
    predictedSteps = 4
    df = pd.concat([x for x in [getComputeDf(data, c, predictedVar, predictedSteps, target) for c in computes] if type(x)!="NoneType"])
    
    df = df.reset_index().drop('index', axis=1)
    features = [x for x in df.columns if x not in ['compute', 'timespan', target]]
        
    X_dfs = []
    y = []
    numberOfSequences = 1
    sequenceSteps = 20
    # generate training data.
    for compute in computes:
        cDf = df[df['compute']==compute]
        if(len(cDf) > sequenceSteps):
            randSteps = np.random.randint(0, len(cDf)-sequenceSteps, numberOfSequences)
            for randStep in randSteps:
                X_dfs.append(cDf.iloc[randStep:randStep+sequenceSteps])
                y.append(X_dfs[-1][target].values[-1])

    X_train_dfs, X_test_dfs, y_train, y_test = train_test_split(X_dfs, y, test_size=0.33)
            
    #Scale data

    # combine the training data to create a scaler
    train_dfs = pd.concat(X_train_dfs)

    scaler = StandardScaler().fit(train_dfs[features].values)
    
    X_train = np.array([scaler.transform(item[features].values) for item in X_train_dfs])
    X_test = np.array([scaler.transform(item[features].values) for item in X_test_dfs])
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    
    return (X_train, X_test, y_train, y_test)

In [9]:
from keras import regularizers
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import Flatten

# from keras import backend as K
# K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=36, inter_op_parallelism_threads=36)))


def createModel(l1Nodes, l2Nodes, d1Nodes, d2Nodes, inputShape):
    # input layer
    lstm1 = LSTM(l1Nodes, input_shape=inputShape, return_sequences=True, kernel_regularizer=regularizers.l2(0.1))
    do1 = Dropout(0.2)
    
    lstm2 = LSTM(l2Nodes, return_sequences=True, kernel_regularizer=regularizers.l2(0.1))
    do2 = Dropout(0.2)
    
    flatten = Flatten()
    
    dense1 = Dense(d1Nodes, activation='relu')
    do3 = Dropout(0.2)
    
    dense2 = Dense(d2Nodes, activation='relu')
    do4 = Dropout(0.2)
    
    # output layer
    outL = Dense(1, activation='relu')
    # combine the layers
#     layers = [lstm1, do1, lstm2, do2, dense1, do3, dense2, do4, outL]
    layers = [lstm1, lstm2, flatten,  dense1, dense2, outL]
    # create the model
    model = Sequential(layers)
    model.compile(optimizer='adam', loss='mse')
    return model

Using TensorFlow backend.


In [18]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.models import load_model
# ten fold
from sklearn.model_selection import KFold
kfold = KFold(n_splits=3, shuffle=True)
from keras.models import load_model
from datetime import datetime

msescores = []

allVariables = ['arrTemperature0', 'arrTemperature1', 'arrTemperature2', 'arrCPU_load0', 'arrMemory_usage0', 'arrFans_health0', 'arrFans_health1', 'arrFans_health2', 'arrFans_health3', 'arrPower_usage0']

start_time = datetime.now()
print(start_time)
for feature in allVariables:
    print(feature)
    X_train, X_test, y_train, Y_test = splitTrainTest(feature)

    counter= 0
    for trainIdx, testIdx in kfold.split(X_train, y_train):
        counter = counter + 1
        # create callbacks
        model_path = feature+'_best_model_fold'+str(counter)+"_"+datetime.now().strftime("%H_%M_%S")+'.h5'
        mc = ModelCheckpoint(model_path, monitor='val_loss', mode='min', verbose=1, save_best_only=True)
        es = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=1)
        # create model
        model = createModel(64, 64, 8, 8, (X_train.shape[1], X_train.shape[2]))
        model.fit(X_train[trainIdx], y_train[trainIdx], validation_data=(X_train[testIdx], y_train[testIdx]), batch_size=32, epochs=40, callbacks=[mc, es])
        # Done load the best model of this fold
        saved_model = load_model(model_path)
        msescores.append({'path': model_path, 'mse': saved_model.evaluate(X_train[testIdx], y_train[testIdx])})
        
end_time = datetime.now()


2019-10-23 18:09:25.736849
arrTemperature0
Train on 208 samples, validate on 104 samples
Epoch 1/40
208/208 [==============================] - 1s 4ms/step - loss: 3511.3299 - val_loss: 4021.1111

Epoch 00001: val_loss improved from inf to 4021.11108, saving model to arrTemperature0_best_model_fold1_18_09_48.h5
Epoch 2/40
208/208 [==============================] - 0s 588us/step - loss: 3490.0395 - val_loss: 3986.8744

Epoch 00002: val_loss improved from 4021.11108 to 3986.87444, saving model to arrTemperature0_best_model_fold1_18_09_48.h5
Epoch 3/40
208/208 [==============================] - 0s 552us/step - loss: 3424.7190 - val_loss: 3896.5302

Epoch 00003: val_loss improved from 3986.87444 to 3896.53018, saving model to arrTemperature0_best_model_fold1_18_09_48.h5
Epoch 4/40
208/208 [==============================] - 0s 569us/step - loss: 3266.8588 - val_loss: 3696.6482

Epoch 00004: val_loss improved from 3896.53018 to 3696.64823, saving model to arrTemperature0_best_model_fold1_18_0

Epoch 26/40
208/208 [==============================] - 0s 598us/step - loss: 58.7488 - val_loss: 51.7130

Epoch 00026: val_loss did not improve from 51.29311
Epoch 27/40
208/208 [==============================] - 0s 568us/step - loss: 57.1699 - val_loss: 49.7711

Epoch 00027: val_loss improved from 51.29311 to 49.77112, saving model to arrTemperature0_best_model_fold2_18_10_04.h5
Epoch 28/40
208/208 [==============================] - 0s 538us/step - loss: 56.7085 - val_loss: 50.4343

Epoch 00028: val_loss did not improve from 49.77112
Epoch 29/40
208/208 [==============================] - 0s 541us/step - loss: 54.2914 - val_loss: 47.8505

Epoch 00029: val_loss improved from 49.77112 to 47.85050, saving model to arrTemperature0_best_model_fold2_18_10_04.h5
Epoch 30/40
208/208 [==============================] - 0s 558us/step - loss: 53.7665 - val_loss: 46.4779

Epoch 00030: val_loss improved from 47.85050 to 46.47787, saving model to arrTemperature0_best_model_fold2_18_10_04.h5
Epoch 31/

208/208 [==============================] - 0s 576us/step - loss: 47.0262 - val_loss: 103.1443

Epoch 00030: val_loss did not improve from 101.87343
Epoch 00030: early stopping
104/104 [==============================] - 0s 1ms/step
arrTemperature1
Train on 208 samples, validate on 104 samples
Epoch 1/40
208/208 [==============================] - 1s 4ms/step - loss: 2273.3446 - val_loss: 2089.0007

Epoch 00001: val_loss improved from inf to 2089.00071, saving model to arrTemperature1_best_model_fold1_18_11_06.h5
Epoch 2/40
208/208 [==============================] - 0s 551us/step - loss: 2234.0167 - val_loss: 2009.9847

Epoch 00002: val_loss improved from 2089.00071 to 2009.98473, saving model to arrTemperature1_best_model_fold1_18_11_06.h5
Epoch 3/40
208/208 [==============================] - 0s 588us/step - loss: 2140.0529 - val_loss: 1855.1820

Epoch 00003: val_loss improved from 2009.98473 to 1855.18204, saving model to arrTemperature1_best_model_fold1_18_11_06.h5
Epoch 4/40
208/208 [

208/208 [==============================] - 0s 577us/step - loss: 45.1271 - val_loss: 36.7346

Epoch 00025: val_loss improved from 37.95290 to 36.73460, saving model to arrTemperature1_best_model_fold2_18_11_22.h5
Epoch 26/40
208/208 [==============================] - 0s 561us/step - loss: 44.6011 - val_loss: 35.8132

Epoch 00026: val_loss improved from 36.73460 to 35.81320, saving model to arrTemperature1_best_model_fold2_18_11_22.h5
Epoch 27/40
208/208 [==============================] - 0s 542us/step - loss: 43.8630 - val_loss: 34.9976

Epoch 00027: val_loss improved from 35.81320 to 34.99759, saving model to arrTemperature1_best_model_fold2_18_11_22.h5
Epoch 28/40
208/208 [==============================] - 0s 578us/step - loss: 42.9719 - val_loss: 35.0327

Epoch 00028: val_loss did not improve from 34.99759
Epoch 29/40
208/208 [==============================] - 0s 542us/step - loss: 42.3289 - val_loss: 34.2907

Epoch 00029: val_loss improved from 34.99759 to 34.29071, saving model to

208/208 [==============================] - 0s 560us/step - loss: 242.3320 - val_loss: 236.5019

Epoch 00012: val_loss improved from 236.80922 to 236.50187, saving model to arrTemperature2_best_model_fold1_18_12_25.h5
Epoch 13/40
208/208 [==============================] - 0s 587us/step - loss: 242.0458 - val_loss: 236.2462

Epoch 00013: val_loss improved from 236.50187 to 236.24617, saving model to arrTemperature2_best_model_fold1_18_12_25.h5
Epoch 14/40
208/208 [==============================] - 0s 585us/step - loss: 241.8080 - val_loss: 236.0342

Epoch 00014: val_loss improved from 236.24617 to 236.03423, saving model to arrTemperature2_best_model_fold1_18_12_25.h5
Epoch 15/40
208/208 [==============================] - 0s 620us/step - loss: 241.6112 - val_loss: 235.8591

Epoch 00015: val_loss improved from 236.03423 to 235.85915, saving model to arrTemperature2_best_model_fold1_18_12_25.h5
Epoch 16/40
208/208 [==============================] - 0s 592us/step - loss: 241.4486 - val_loss

...

Epoch 39/40
208/208 [==============================] - 0s 548us/step - loss: 23982864.4615 - val_loss: 21004889.8462

Epoch 00039: val_loss improved from 23052345.38462 to 21004889.84615, saving model to arrFans_health3_best_model_fold2_18_23_34.h5
Epoch 40/40
208/208 [==============================] - 0s 543us/step - loss: 21927004.0000 - val_loss: 19041209.2308

Epoch 00040: val_loss improved from 21004889.84615 to 19041209.23077, saving model to arrFans_health3_best_model_fold2_18_23_34.h5
104/104 [==============================] - 0s 1ms/step
Train on 208 samples, validate on 104 samples
Epoch 1/40
208/208 [==============================] - 1s 4ms/step - loss: 83406982.7692 - val_loss: 88009845.5385

Epoch 00001: val_loss improved from inf to 88009845.53846, saving model to arrFans_health3_best_model_fold3_18_24_04.h5
Epoch 2/40
208/208 [==============================] - 0s 560us/step - loss: 83382498.4615 - val_loss: 87966393.2308

Epoch 00002: val_loss improved from 88009845.5384

Epoch 32/40
208/208 [==============================] - 0s 556us/step - loss: 56620977.8462 - val_loss: 59404836.0000

Epoch 00032: val_loss improved from 61259349.53846 to 59404836.00000, saving model to arrFans_health3_best_model_fold3_18_24_04.h5
Epoch 33/40
208/208 [==============================] - 0s 553us/step - loss: 54808047.6923 - val_loss: 57489641.8462

Epoch 00033: val_loss improved from 59404836.00000 to 57489641.84615, saving model to arrFans_health3_best_model_fold3_18_24_04.h5
Epoch 34/40
208/208 [==============================] - 0s 561us/step - loss: 52908994.1538 - val_loss: 55512439.3846

Epoch 00034: val_loss improved from 57489641.84615 to 55512439.38462, saving model to arrFans_health3_best_model_fold3_18_24_04.h5
Epoch 35/40
208/208 [==============================] - 0s 544us/step - loss: 51000654.7692 - val_loss: 53493613.2308

Epoch 00035: val_loss improved from 55512439.38462 to 53493613.23077, saving model to arrFans_health3_best_model_fold3_18_24_04.h5
Epoc


Epoch 00026: val_loss improved from 143.16973 to 139.61322, saving model to arrPower_usage0_best_model_fold1_18_24_57.h5
Epoch 27/40
208/208 [==============================] - 0s 551us/step - loss: 142.7188 - val_loss: 135.4741

Epoch 00027: val_loss improved from 139.61322 to 135.47414, saving model to arrPower_usage0_best_model_fold1_18_24_57.h5
Epoch 28/40
208/208 [==============================] - 0s 555us/step - loss: 139.6511 - val_loss: 134.6700

Epoch 00028: val_loss improved from 135.47414 to 134.67005, saving model to arrPower_usage0_best_model_fold1_18_24_57.h5
Epoch 29/40
208/208 [==============================] - 0s 563us/step - loss: 137.9491 - val_loss: 132.3700

Epoch 00029: val_loss improved from 134.67005 to 132.36998, saving model to arrPower_usage0_best_model_fold1_18_24_57.h5
Epoch 30/40
208/208 [==============================] - 0s 560us/step - loss: 134.0435 - val_loss: 130.9986

Epoch 00030: val_loss improved from 132.36998 to 130.99862, saving model to arrPowe

208/208 [==============================] - 0s 538us/step - loss: 90.3219 - val_loss: 165.2366

Epoch 00023: val_loss improved from 169.00926 to 165.23655, saving model to arrPower_usage0_best_model_fold2_18_25_28.h5
Epoch 24/40
208/208 [==============================] - 0s 608us/step - loss: 86.5451 - val_loss: 162.9280

Epoch 00024: val_loss improved from 165.23655 to 162.92801, saving model to arrPower_usage0_best_model_fold2_18_25_28.h5
Epoch 25/40
208/208 [==============================] - 0s 567us/step - loss: 83.1460 - val_loss: 160.7989

Epoch 00025: val_loss improved from 162.92801 to 160.79892, saving model to arrPower_usage0_best_model_fold2_18_25_28.h5
Epoch 26/40
208/208 [==============================] - 0s 552us/step - loss: 80.2768 - val_loss: 158.7148

Epoch 00026: val_loss improved from 160.79892 to 158.71485, saving model to arrPower_usage0_best_model_fold2_18_25_28.h5
Epoch 27/40
208/208 [==============================] - 0s 540us/step - loss: 78.2007 - val_loss: 157

208/208 [==============================] - 0s 566us/step - loss: 201.7346 - val_loss: 174.4136

Epoch 00020: val_loss did not improve from 166.69669
Epoch 21/40
208/208 [==============================] - 0s 559us/step - loss: 196.5427 - val_loss: 169.3847

Epoch 00021: val_loss did not improve from 166.69669
Epoch 22/40
208/208 [==============================] - 0s 599us/step - loss: 189.4454 - val_loss: 169.0086

Epoch 00022: val_loss did not improve from 166.69669
Epoch 00022: early stopping
104/104 [==============================] - 0s 1ms/step


In [19]:
print ('start time: ', start_time)
print ('end time:', end_time)

start time:  2019-10-23 18:09:25.736849
end time: 2019-10-23 18:26:28.002689


In [12]:
# Plot and see data distribution

from matplotlib import pyplot as plt
import seaborn as sns

def plotAttrDataOfId(data, compute, features):
    plt.figure(figsize=(30, 20))
    for i, v in enumerate(features):
        plt.subplot(10, 3, i+1)
        cDf = df[df['compute']==compute]
        plt.plot(cDf['timespan'], cDf[v])
        plt.title(v)
        plt.tight_layout()

In [13]:
# for x in np.random.randint(0, len(computes), 3):
#     plotAttrDataOfId(df, computes[x], features)

In [14]:
def plotDataDistribution(data, features):
    plt.figure(figsize=(30, 10))
    for i, v in enumerate(features):
        plt.subplot(3, 10, i+1)
        sns.distplot(list(data[v].values))
        plt.title(v)
    plt.tight_layout()

In [ ]:
# plotDataDistribution(df, features)

In [15]:
from datetime import datetime
print(datetime.now().strftime("%H_%M_%S"))

17_54_25


In [ ]:
sns.distplot(y_train)

In [ ]:
sns.distplot(y_test)

In [ ]:
print (type(X_dfs[0]))